In [1]:

import json
import os
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


# Configuración del dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Crear un diccionario para mapear las posiciones a índices
position_to_index = {
    '5050_guard': 0,
    'back1': 1,
    'back2': 2,
    'closed_guard1': 3,
    'closed_guard2': 4,
    'half_guard1': 5,
    'half_guard2': 6,
    'mount1': 7,
    'mount2': 8,
    'open_guard1': 9,
    'open_guard2': 10,
    'side_control1': 11,
    'side_control2': 12,
    'standing': 13,
    'takedown1': 14,
    'takedown2': 15,
    'turtle1': 16,
    'turtle2': 17
}

MemoryError: 

In [ ]:
def load_annotations(file_path):
    """
    Carga las anotaciones desde un archivo JSON.
    """
    with open(file_path, 'r') as f:
        return json.load(f)

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, annotations, seq_length=10, position_to_index=None):
        """
        Inicializa el dataset para análisis de secuencias.
        """
        self.annotations = annotations
        self.seq_length = seq_length
        self.position_to_index = position_to_index

    def __len__(self):
        """
        Devuelve el número de secuencias disponibles.
        """
        return len(self.annotations) - self.seq_length

    def __getitem__(self, idx):
        """
        Devuelve una secuencia y su etiqueta correspondiente.
        """
        sequence = [
            self.annotations[idx + i]['Pose1'] + self.annotations[idx + i]['Pose2']
            for i in range(self.seq_length)
        ]
        sequence = torch.tensor(sequence).view(self.seq_length, -1)
        label_str = self.annotations[idx + self.seq_length - 1]['Position']
        label = self.position_to_index[label_str]
        return sequence, torch.tensor(label).long()

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        """
        Inicializa el modelo LSTM.
        """
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        Define la pasada hacia adelante del modelo.
        """
        lstm_out, _ = self.lstm(x)
        final_output = lstm_out[:, -1, :]
        return self.fc(final_output)

In [ ]:
def train_and_validate_lstm(train_loader, val_loader, model, criterion, optimizer, num_epochs=25, patience=5):
    """
    Entrena y valida el modelo LSTM, aplicando early stopping.
    """
    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}')

        # Validación
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(val_loader.dataset)
        print(f'Validation Loss: {val_loss}')

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_lstm_model.pth')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping")
                break

        torch.cuda.empty_cache()

In [ ]:
def cross_validation_kfold_lstm(dataset, k_folds=5, batch_size=16, num_epochs=25, patience=5):
    """
    Realiza validación cruzada con K-Fold para el modelo LSTM.
    """
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f'Fold {fold+1}/{k_folds}')

        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size)

        input_size = 102  # 34 puntos clave x 3 coordenadas
        hidden_size = 128
        output_size = len(position_to_index)

        model = LSTMModel(input_size, hidden_size, output_size).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        train_and_validate_lstm(train_loader, val_loader, model, criterion, optimizer, num_epochs=num_epochs, patience=patience)


In [ ]:
def main():
    """
    Función principal para ejecutar el flujo de validación cruzada.
    """
    annotations_path = '../mnt/V2/annotations/annotations_preprocessed.json'
    annotations = load_annotations(annotations_path)

    seq_length = 10
    dataset = SequenceDataset(annotations, seq_length=seq_length, position_to_index=position_to_index)

    cross_validation_kfold_lstm(dataset, k_folds=5, batch_size=16, num_epochs=25, patience=5)

if __name__ == '__main__':
    main()

Fold 1/5
Epoch 1/25, Loss: 1.6488970523677138
Validation Loss: 0.9756716276733587
Epoch 2/25, Loss: 0.8425435898064693
Validation Loss: 0.7088313145651822
Epoch 3/25, Loss: 0.6011588091341883
Validation Loss: 0.5387424555231071
Epoch 4/25, Loss: 0.4760379673103843
Validation Loss: 0.4994262053732794
Epoch 5/25, Loss: 0.39633889353972906
Validation Loss: 0.3866312280918368
Epoch 6/25, Loss: 0.3466005036771973
Validation Loss: 0.38201721466235533
Epoch 7/25, Loss: 0.3005061926852468
Validation Loss: 0.43790605267750177
Epoch 8/25, Loss: 0.2639035658116498
Validation Loss: 0.29971387562015606
Epoch 9/25, Loss: 0.22537042306709404
Validation Loss: 0.30514774557728497
Epoch 10/25, Loss: 0.1997382116440973
Validation Loss: 0.310759763855384
Epoch 11/25, Loss: 0.16685377822946054
Validation Loss: 0.31460911278010767
Epoch 12/25, Loss: 0.145830579540528
Validation Loss: 0.2864063510366303
Epoch 13/25, Loss: 0.12097696600548324
Validation Loss: 0.34249666031100984
Epoch 14/25, Loss: 0.100951654